# 00 · Basic Tree (Programmatic Mesh → Grow → Activate → Export)

**Goal.** Minimal, self-contained demo that:
1) Generates a small surface mesh programmatically (sphere),
2) Grows a fractal tree on it,
3) Wraps it into a `PurkinjeTree` and runs FIM activation,
4) Exports VTK files for visualization.

**Why start here?** No external data is required. This notebook is the fastest path to confirm the pipeline works end-to-end on your machine.

**Outputs**
- `output/examples/00_basic_tree/basic_tree_AT.vtu` — unstructured grid with activation time field.
- `output/examples/00_basic_tree/basic_tree_meshio.vtu` — line mesh via `meshio` (compatible with ParaView).
- `output/examples/00_basic_tree/basic_tree_pmj.vtp` — PMJs as a point set.

You can keep runtimes small with `EXAMPLES_LITE=1` (default in this notebook).


In [ ]:
%pip install -q --upgrade pip
%pip install -q purkinje-uv

## Settings, Reproducibility, and Output Paths

We use:
- `SEED` for determinism where applicable.
- `LITE` toggle to keep the example small and fast by default.
- A dedicated `OUT_DIR` under `output/examples/00_basic_tree/` to avoid clutter.

You may override these with environment variables:
- `EXAMPLES_SEED=2025`
- `EXAMPLES_LITE=0`
- `EXAMPLES_SHOW=1` (optional 3D viewer at the end)


In [ ]:
from pathlib import Path
import os
import numpy as np
import pyvista as pv
from purkinje_uv import FractalTreeParameters, FractalTree, PurkinjeTree

In [ ]:
SEED = int(os.getenv("EXAMPLES_SEED", "1234"))
LITE = bool(int(os.getenv("EXAMPLES_LITE", "1")))
SHOW = bool(int(os.getenv("EXAMPLES_SHOW", "0")))  # optional PyVista preview

OUT_DIR = Path("output") / "examples" / "00_basic_tree"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

np.random.seed(SEED)
print(f"SEED={SEED}  LITE={LITE}  SHOW={SHOW}")
print("OUT_DIR:", OUT_DIR)


## Create a Programmatic Mesh (Sphere)

We generate a simple triangulated sphere with PyVista and save it as an OBJ under `data/`.
This avoids any external files and ensures the example is portable.

In [ ]:
mesh_path = DATA_DIR / "basic_sphere.obj"
sphere = pv.Sphere(radius=1.0, theta_resolution=96, phi_resolution=96).triangulate()

# Save via PyVista; if your environment lacks the OBJ writer for some reason,
# we fall back to meshio.
try:
    sphere.save(mesh_path)
    saved_via = "pyvista"
except Exception as e:
    print("PyVista save failed, falling back to meshio:", repr(e))
    import meshio
    # meshio expects faces as a list of arrays; convert polydata to triangles
    faces = sphere.faces.reshape(-1, 4)[:, 1:]  # (n_cells, 3)
    meshio.write(
        mesh_path,
        meshio.Mesh(points=sphere.points, cells=[("triangle", faces)]),
    )
    saved_via = "meshio"

print(f"Saved mesh: {mesh_path}  | points={sphere.n_points}  faces={sphere.n_cells}  via={saved_via}")

## Fractal Tree Parameters (Small & CI-friendly)

We start with conservative values that run quickly:
- `l_segment=0.01`
- `init_length=0.25`, `length=0.12` (initial and subsequent segment lengths)
- `branch_angle=0.15 rad`, `w=0.1` (branching control)
- `init_node_id=0`, `second_node_id=1` (work well on the generated sphere)

If you disable LITE mode, you can increase size/complexity later.

In [ ]:
def make_params(path: Path) -> FractalTreeParameters:
    p = FractalTreeParameters(
        meshfile=str(path),
        l_segment=0.01,
        init_length=0.25,
        length=0.12,
        branch_angle=0.15,
        w=0.1,
        init_node_id=0,
        second_node_id=1,
        N_it=5,
    )
    return p

params = make_params(mesh_path)
params

## Grow the Fractal Tree

We instantiate `FractalTree` with our parameters and call `grow_tree()`.
The class loads the OBJ, performs the internal UV mapping/scaling, and generates:
- `nodes_xyz` (list of 3D points),
- `connectivity` (list of edges),
- `end_nodes` (PMJ indices).

In [ ]:
ft = FractalTree(params=params)
ft.grow_tree()

n_nodes = len(ft.nodes_xyz)
n_edges = len(ft.connectivity)
n_pmj   = len(ft.end_nodes)

print(f"Tree grown.")
print(f"Nodes={n_nodes}  Edges={n_edges}  PMJs={n_pmj}")

## Wrap into `PurkinjeTree` and Run FIM Activation

We now build a `PurkinjeTree` and run the Fast Iterative Method (FIM).
- We stimulate node **0** at time **0.0**.
- `return_only_pmj=False` returns activation times for all nodes.

You can inspect PMJ activation times via `act[P.pmj]`.

In [ ]:
P = PurkinjeTree(
    nodes=np.asarray(ft.nodes_xyz, dtype=float),
    connectivity=np.asarray(ft.connectivity, dtype=int),
    end_nodes=np.asarray(ft.end_nodes, dtype=int),
)

act = P.activate_fim(
    x0=np.array([0], dtype=int),
    x0_vals=np.array([0.0], dtype=float),
    return_only_pmj=False,
)

act_min = float(np.min(act)) if act.size else float("nan")
act_max = float(np.max(act)) if act.size else float("nan")

print("Activation array:", act.shape, " min/max:", act_min, act_max)
if hasattr(P, "pmj"):
    pmj_count = len(getattr(P, "pmj"))
    print("PMJ count:", pmj_count)


## Export for Visualization

We save three files into `OUT_DIR`:
- `*_AT.vtu` — Unstructured grid with activation times (native writer).
- `*_meshio.vtu` — Line mesh via `meshio`.
- `*_pmj.vtp` — PMJs as points.

Open them in ParaView to verify visually.

In [ ]:
at_path    = OUT_DIR / "basic_tree_AT.vtu"
meshio_path= OUT_DIR / "basic_tree_meshio.vtu"
pmj_path   = OUT_DIR / "basic_tree_pmj.vtp"

P.save(str(at_path))
P.save_meshio(str(meshio_path))
P.save_pmjs(str(pmj_path))

print("Wrote:")
print(" -", at_path)
print(" -", meshio_path)
print(" -", pmj_path)


## Optional: Quick 3D Preview (PyVista)

If you set `EXAMPLES_SHOW=1`, the cell below displays:
- Tree lines,
- PMJ points.

This is optional and may require a local 3D backend.

In [ ]:
if SHOW:
    import pyvista as pv
    pts = np.asarray(ft.nodes_xyz)
    edges = np.asarray(ft.connectivity)

    # Build a PolyData with polyline cells
    # VTK "lines" connectivity format: [n, i0, i1, n, j0, j1, ...]
    lines = np.hstack([[2, e[0], e[1]] for e in edges]).astype(np.int32)

    poly = pv.PolyData()
    poly.points = pts
    poly.lines = lines

    pl = pv.Plotter()
    pl.add_mesh(poly, line_width=2)
    if len(ft.end_nodes) > 0:
        pl.add_points(pts[np.asarray(ft.end_nodes, dtype=int)], point_size=8)
    pl.show()
else:
    print("Preview disabled. Set EXAMPLES_SHOW=1 to enable.")


## Verification Checklist

- This notebook produced three files in `output/examples/00_basic_tree/`.
- Activation array has the same length as the number of tree nodes.
- PMJ file exists and contains points.
- Runs in under a couple of minutes with `EXAMPLES_LITE=1`.

**Reproducibility tips**
- Keep `SEED` fixed to stabilize stochastic parts.
- Use `EXAMPLES_LITE=1` for CI or quick runs; set `EXAMPLES_LITE=0` to explore larger trees.

## Troubleshooting

- **`vtk` wheel installation errors (Windows/Linux):** ensure you’re using a supported Python version and recent pip (`python -m pip install --upgrade pip`).
- **`ImportError: purkinje_uv …`:** the notebook auto-installs the local repo in editable mode if it finds `pyproject.toml`. If your code lives outside the working directory, install it manually (`pip install -e <path-to-repo>`).
- **OBJ writer not available:** the notebook falls back to `meshio` to write OBJ or VTU. Ensure `meshio` is installed (the first cell does this).
- **Blank preview window:** set `EXAMPLES_SHOW=1` to enable the viewer and ensure your environment supports interactive windows (e.g., not running headless).
